In [ ]:
from scipy._lib._test_ccallback import test_get_data_capsule

from models.graph_transformers.SAT.sat import models
from models.graph_transformers.SAT.sat import layers
import torch
import torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader


def ptr_to_complete_edge_index(ptr):
    from_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat_interleave(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    to_lists = [torch.arange(ptr[i], ptr[i + 1]).repeat(ptr[i + 1] - ptr[i]) for i in range(len(ptr) - 1)]
    combined_complete_edge_index = torch.vstack((torch.cat(from_lists, dim=0), torch.cat(to_lists, dim=0)))
    return combined_complete_edge_index
test_data = torch.FloatTensor([1., 2., 3., 4., 5.]).unsqueeze(1)
edge_index = torch.LongTensor([[0,1,2],[1, 2, 3]])

test_graph = Data(x=test_data, edge_index=edge_index)
test_data2 = torch.FloatTensor([-1., -2., -3., -4., 10]).unsqueeze(1)
edge_index2 = torch.LongTensor([[0,1],[1, 2]])
test_graph2 = Data(x=test_data2, edge_index=edge_index2)
loader = iter(DataLoader([test_graph, test_graph2], batch_size=2))
batch = next(loader)


batch.complete_edge_index = torch.LongTensor([[0,1,5],[1,2,6]])
# batch.complete_edge_index = ptr_to_complete_edge_index(batch.ptr)


model = models.GraphTransformer(in_size=1,
                                num_class=1,
                                in_embed=False,
                                d_model=3,
                                dim_feedforward=2,
                                num_heads=1,
                                num_layers=1,
                                abs_pe=False,
                                abs_pe_dim=None,

                                use_edge_attr=False,
                                num_edge_features=0,
                                dropout=0.,
                                k_hop=2)


#model(batch)

In [ ]:
amrlayer = layers.AttentionAMR(1, 0, 1, 0, True).cuda()

In [ ]:
batch.ptr

In [ ]:
edge_index_source = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[0][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()
edge_index_target = torch.LongTensor([[i for i in torch.arange(batch.edge_index.shape[1])], [batch.edge_index[1][i] for i in torch.arange(batch.edge_index.shape[1])]]).cuda()

amrlayer(batch.x.cuda(), batch.x.cuda(), batch.edge_index.cuda(), edge_index_source, edge_index_target, ptr=batch.ptr.cuda())

In [ ]:
embed = torch.nn.Linear(1000, 128)

In [ ]:
import pickle
with open("data/hol4/data/torch_graph_dict.pk", "rb") as f:
    tg = pickle.load(f)
test_graph_1 = tg[list(tg.keys())[0]]
test_graph_1
test_graph_2 = tg[list(tg.keys())[2]]
test_graph_2
loader = iter(DataLoader([test_graph_1, test_graph_2], batch_size=2))
batch = next(loader)

In [ ]:

from models.graph_transformers.SAT.sat import layers
amrlayer = layers.AttentionAMR(128,0,2,0,True)

x = embed(batch.x)

out_s, out_t = amrlayer(x, x, batch.edge_index)

In [ ]:
amrlayer(out_s, out_t, batch.edge_index)

In [ ]:
batch.edge_index

In [ ]:

batch.edge_attr = batch.edge_attr.long()
model = models.AMRTransformer(in_size=1000,num_class=0,d_model=128,num_heads=4,dim_feedforward=128,dropout=0.1,use_edge_attr=True,num_layers=1,layer_norm=True,in_embed=False).cuda()
model(batch.cuda())

In [ ]:
torch.tensor([1.,2.,3.,4.,5.]).to(torch.tensor([1.,1.,2.,5,5234,643,2345]))

In [ ]:
batch.x

In [2]:
import torch
from models.graph_transformers.SAT.sat.layers import AttentionRelations
model = AttentionRelations(embed_dim=8, edge_dim=32)
x = torch.rand(11, 8)
edge_index = torch.LongTensor([[0,1,2,3,4,9,8,7,6,8],[4,3,2,1,0,7,8,8,6,6]])
edge_attr = torch.LongTensor([0,0,1,3,0,1,2,3,2,1])
softmax_idx = [5]
model(x, edge_index, softmax_idx, edge_attr)

torch.Size([10, 8])
cls
torch.Size([1, 2, 8])
R
torch.Size([5, 2, 8])


torch.Size([2, 8])

In [9]:
torch.tensor_split(x, [0,5,10])

(tensor([], size=(0, 8)),
 tensor([[0.9918, 0.1399, 0.9252, 0.5609, 0.2900, 0.4016, 0.1394, 0.7816],
         [0.0530, 0.7915, 0.4800, 0.6560, 0.4102, 0.3130, 0.6216, 0.5257],
         [0.8181, 0.1945, 0.2962, 0.6820, 0.1563, 0.4805, 0.4853, 0.2181],
         [0.2078, 0.6373, 0.8061, 0.9837, 0.1156, 0.7052, 0.2368, 0.8982],
         [0.6767, 0.6834, 0.0777, 0.7179, 0.4268, 0.3100, 0.6686, 0.3017]]),
 tensor([[0.5775, 0.3654, 0.7138, 0.3392, 0.1116, 0.3705, 0.6395, 0.7384],
         [0.1987, 0.1943, 0.5168, 0.1664, 0.3289, 0.7054, 0.9939, 0.4983],
         [0.0591, 0.2129, 0.7034, 0.7597, 0.5017, 0.4793, 0.8638, 0.4914],
         [0.9856, 0.4725, 0.5217, 0.5346, 0.4938, 0.3722, 0.4069, 0.9200],
         [0.0018, 0.2812, 0.2255, 0.5144, 0.7758, 0.5511, 0.2027, 0.8557]]),
 tensor([[0.0227, 0.6568, 0.1133, 0.3209, 0.8643, 0.5390, 0.9473, 0.7103]]))